# **Vehicle Counter** 

In [2]:
# Import Some Library
import cv2 as cv
import numpy as np

In [21]:
# web capture
capture = cv.VideoCapture('video.mp4')

count_line_postion = 550

min_width_rectangle =  15
min_length_rectangle = 15

# counting list
def center_handle(x,y,w,h):
    x1 = int(w/2)
    y1=  int(h/2)
    cx=x+x1
    cy=y+y1
    return cx,cy

detect = [] 
offset = 6 # allowable error between pixcel
counter = 0

# Subtract Background
bg_sub = cv.createBackgroundSubtractorMOG2(history=500, varThreshold=350, detectShadows=False)
while True:

    ret,frame = capture.read()
    # convert into grey
    grey = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)

    # median filter
    n=20
    kernel = np.array([[2/n,4/n,2/n],
                       [4/n,8/n,4/n],
                       [2/n,4/n,2/n]])

    blur = cv.filter2D(grey,0,kernel)
    
    # blur = cv.GaussianBlur(grey,(3,3),0)
    #apply on each frame
    img_sub = bg_sub.apply(blur)
    kernel = np.array([[1,1,1],
                       [1,1,1],
                       [1,1,1]],dtype=np.uint8)
  # horizontal Image
    dilat = cv.dilate(img_sub,kernel,iterations = 12)  #12    
    
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(4,4))
    
    kernel = np.array([[1,1,1],
                       [1,1,1],
                       [1,1,1]])
      # horizontal Image
    diltada = cv.erode(dilat,kernel,iterations = 28) #28

    # diltada = cv.morphologyEx(dilat,cv.MORPH_ELLIPSE,kernel)   
    # diltada = cv.morphologyEx(diltada,cv.MORPH_CLOSE,kernel)
    counterShape,h = cv.findContours(diltada,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)
    
    cv.line(frame,(50,count_line_postion),(1500,count_line_postion),(255,0,255),2)
    
    for (i,c) in enumerate(counterShape):
        (x,y,w,h) = cv.boundingRect(c)
        validate_counter = (w>= min_width_rectangle) and (w>= min_length_rectangle)   
        if not validate_counter:
            continue
        
        cv.rectangle(frame,(x-5,y-5),(x+w-5,y+w-5),(0,255,0),2)
        # cv.putText(frame,"Vehicle Counter :"+str(counter),(x,y-20),cv.FONT_HERSHEY_SIMPLEX,1,(125,245,255),2)    
        
        center = center_handle(x,y,w,h)
        detect.append(center)
    # Assuming detect is a list containing the center coordinates (x, y)
        cv.circle(frame, center, 5, (0, 0, 255), -1)
        
        for (x,y) in detect:
            if y<(count_line_postion + offset ) and y>(count_line_postion - offset ):
                counter += 1
                cv.line(frame,(50,count_line_postion),(1500,count_line_postion),(0,115,205),2)
                detect.remove((x,y))
                # print('vehcile Counter :'+str(counter))
    
    cv.putText(frame,"Vehicle Counter :"+str(counter),(450,75),cv.FONT_HERSHEY_SIMPLEX,2,(0,0,255),2)    
                
 
    cv.imshow('Video Original',frame)    
    if cv.waitKey(1) == 13:
        break

cv.destroyAllWindows()

capture.release

<function VideoCapture.release>